In [ ]:
import numpy as np
import msmrd2
import msmrd2.tools.quaternions as quaternions
import random
import msmrd2.visualization as msmrdvis

In [ ]:
# Main parameters 
numparticles = 2
D = 1.0
Drot = 1.0
initialState = 3 #'B'# 'A'
boxsize = 6

# Parameters related to patchy Particle potential with angular dependence
sigma = 1.0
angleDiff = 3*np.pi/5.0
patch1 = np.array([np.cos(angleDiff/2),np.sin(angleDiff/2),0.])
patch2 = np.array([np.cos(-angleDiff/2),np.sin(-angleDiff/2),0.])
patchesCoordinates = [patch1, patch2]

# Bound states
boundStatesA = [1, 2, 5, 6] # U-shaped bound dimer, corresponds to A state
boundStatesB = [3, 4, 7, 8] # Zigzag-shaped bound dimer, corresponds to B state

In [ ]:
# Generate particle list with two particle dimers in bound state
def generateParticleList(state, boxsize, D, Drot):
    '''
    Generate a random particle list of two particles corresponding to either state A or state B. As each
    state has several different configurations, this functions picks one randomly. The definition of the
    states is taken form patchyDimer.cpp
    :param state: bound state in which the two particle list should be in, A or B
    :param D and Drot: diffusion coefficients of particles.
    :param randomSeed: seed for python random generator. Important to specify in parallel runs. Default value of -1
    will use the default seed.
    :return: random particle list corresponding to either state A or state B.
    '''

    # Transform boxsize to vector if neccesarry.
    if np.isscalar(boxsize):
        boxsize = np.array([boxsize, boxsize, boxsize])

    position1 = np.array([0.0, 0.0, 0.0])
    #position1 = np.array([boxsize[0]*random.random()-0.5*boxsize[0],
    #                      boxsize[1]*random.random()-0.5*boxsize[1],
    #                      boxsize[2]*random.random()-0.5*boxsize[2]])
    orientation1 = np.array([1.0, 0.0, 0.0, 0.0])
    # Define relative position
    relpos1 = np.array([np.cos(angleDiff / 2.0),np.sin(angleDiff / 2.0), 0])
    relpos2 = np.array([np.cos(angleDiff / 2.0),np.sin(-1*angleDiff / 2.0), 0])
    relPos1orthogonal = np.array([-1.0 * np.sin(angleDiff / 2.0), np.cos(angleDiff / 2.0), 0.0])
    relPos2orthogonal = np.array([np.sin(angleDiff / 2.0), np.cos(angleDiff / 2.0), 0.0])
    # Define relative rotations
    rotations = [None]*8
    rotations[0] = np.pi * relPos1orthogonal
    rotations[1] = np.array([0.0, 0.0, -2 * np.pi / 5.0])
    rotations[2] = np.array([0.0, 0.0, np.pi])
    rotations[3] = np.array([0.0, np.pi, 0.0])
    # --first 4 rotations correspond to binding on top patch of particle 1, next 4 rotations to bottom patch
    rotations[4] = np.pi * relPos2orthogonal
    rotations[5] = np.array([0.0, 0.0, 2 * np.pi / 5.0])
    rotations[6] = np.array([0.0, 0.0, np.pi])
    rotations[7] = np.array([0.0, np.pi, 0.0])
    # Convert axis-angle rotations to quaternions
    quatRotations = [None]*8
    for i in range(8):
        quatRotations[i] = quaternions.angle2quat(rotations[i])
    # Assign correct position2 depending on the states
    pos2 = [None]*8
    pos2[0:4] = [position1 + relpos1]*4
    pos2[4:8] = [position1 + relpos2]*4

    # Assign position and orientation depending on initial state A or B
    if state in boundStatesA or state in boundStatesB:
        substate = state
        position2 = pos2[substate - 1]
        orientation2 = quatRotations[substate - 1]
    elif state == 'A':
        substate = random.choice([1,2,5,6])
        position2 = pos2[substate - 1]
        orientation2 = quatRotations[substate - 1]
    elif state == 'B':
        substate = random.choice([3,4,7,8])
        position2 = pos2[substate - 1]
        orientation2 = quatRotations[substate - 1]
    part1 = msmrd2.particle(D, Drot, position1, orientation1)
    part2 = msmrd2.particle(D, Drot, position2, orientation2)
    partlist = msmrd2.integrators.particleList([part1, part2])
    return partlist

In [ ]:
# Generate list of particle lists (to plot several bound couples at the same time)
numPartLists = 1 #20
partlists = [None]*numPartLists
for i in range(numPartLists):
    partlists[i] = generateParticleList(initialState, boxsize, D, Drot)

In [ ]:
# Print particlelist into file and generates plotting tcl file for VMD
timesteps = 500000 #20000000
stride = 1000 #250 #1000
datafile  = open('../../data/vmd/testInitialCondition.xyz', 'w')
datafile.write(str(6*len(partlists)) + '\n')
datafile.write(str(0) + '\n')
for i in range(numPartLists):
    for j, part in enumerate(partlists[i]):
        v0 = part.position
        v1 = v0 + 0.45*sigma*quaternions.rotateVec(patchesCoordinates[0], part.orientation)
        v2 = v0 + 0.45*sigma*quaternions.rotateVec(patchesCoordinates[1], part.orientation)
        datafile.write('type_0' + ' ' + ' '.join(map(str, v0)) + '\n')
        datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
        datafile.write('type_1' + ' ' + ' '.join(map(str, v2)) + '\n')
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyParticles(numparticles = numparticles*numPartLists, 
                                     outfname = "testInitialCondition", 
                                     tclfname = "../../data/vmd/testInitialCondition2vmd.tcl")
print("Percentage complete: ", 100, " %")

In [ ]:
# Print particlelist into file and generates plotting tcl file for VMD (multicolor version)
timesteps = 500000 #20000000
stride = 1000 #250 #1000
datafile  = open('../../data/vmd/testInitialCondition.xyz', 'w')
datafile.write(str(6*len(partlists)) + '\n')
datafile.write(str(0) + '\n')
for i in range(numPartLists):
    for j, part in enumerate(partlists[i]):
        v0 = part.position
        v1 = v0 + 0.45*sigma*quaternions.rotateVec(patchesCoordinates[0], part.orientation)
        v2 = v0 + 0.45*sigma*quaternions.rotateVec(patchesCoordinates[1], part.orientation)
        datafile.write('type_0' + str(j) + ' ' + ' '.join(map(str, v0)) + '\n')
        datafile.write('type_1' + ' ' + ' '.join(map(str, v1)) + '\n')
        datafile.write('type_2' + ' ' + ' '.join(map(str, v2)) + '\n')
datafile.close()
# Generate TCL script to visualize with VMD
msmrdvis.generateTCL_patchyParticlesMultiColor(numparticles = numparticles*numPartLists, 
                                     outfname = "testInitialCondition", 
                                     tclfname = "../../data/vmd/testInitialCondition2vmd.tcl")
print("Percentage complete: ", 100, " %")